In [1]:
import itertools
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 
from torch.autograd import Variable

from torchvision import transforms

import numpy as np
import pandas as pd

%matplotlib inline

In [5]:
df_label = pd.read_csv('./Platin_Data/Semi_Final/common_patient.csv', sep = '\t', index_col = 0)

In [6]:
df_label

,PLATINUM_STATUS
PATIENT_ID,
TCGA-04-1331,Sensitive
TCGA-04-1332,Sensitive
TCGA-04-1347,Sensitive
TCGA-04-1362,Resistant
TCGA-04-1364,Resistant
...,...
TCGA-61-2098,Sensitive
TCGA-61-2109,Sensitive
TCGA-61-2110,Resistant


In [7]:
df_label.loc[df_label['PLATINUM_STATUS'] == 'Sensitive', 'label'] = 0
df_label.loc[df_label['PLATINUM_STATUS'] == 'Resistant', 'label'] = 1

In [8]:
df_label

,PLATINUM_STATUS,label
PATIENT_ID,,
TCGA-04-1331,Sensitive,0.0
TCGA-04-1332,Sensitive,0.0
TCGA-04-1347,Sensitive,0.0
TCGA-04-1362,Resistant,1.0
TCGA-04-1364,Resistant,1.0
...,...,...
TCGA-61-2098,Sensitive,0.0
TCGA-61-2109,Sensitive,0.0
TCGA-61-2110,Resistant,1.0


In [12]:
df_data = pd.read_csv('./Platin_Data/Semi_Final/PC_tpm_250_log.csv', sep = '\t', index_col = 0)

In [13]:
df_data

,TCGA-04-1331,TCGA-04-1332,TCGA-04-1347,TCGA-04-1362,TCGA-04-1364,TCGA-04-1365,TCGA-04-1514,TCGA-04-1530,TCGA-04-1536,TCGA-04-1542,...,TCGA-61-2000,TCGA-61-2008,TCGA-61-2009,TCGA-61-2092,TCGA-61-2097,TCGA-61-2098,TCGA-61-2109,TCGA-61-2110,TCGA-61-2111,TCGA-61-2113
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000131096,-6.366282,-4.488628,-7.601309,-4.173178,-2.754576,-6.551323,-3.519839,4.837497,-4.235542,-4.968484,...,-8.525348,-6.816355,-6.354592,-5.572676,-4.620647,-3.424341,-6.197002,-6.249893,2.055320,-6.890379
ENSG00000187581,-0.442050,-3.216314,-3.024444,-1.696708,6.288751,-0.008033,1.893721,-9.965784,-1.684334,0.541251,...,-2.135552,-1.936841,4.180240,-1.913023,-1.649955,5.523327,0.155893,-1.092131,4.107675,-0.801170
ENSG00000047936,0.566665,-2.666762,-8.117399,-4.539869,-8.817501,-0.929409,-7.235335,-3.298323,-8.099063,-6.139259,...,-9.248044,-7.028077,-5.286896,-9.965784,-6.774663,-3.943922,-4.107782,-7.348430,-8.332655,-7.245962
ENSG00000186198,-2.268812,-1.536532,-2.476620,-3.195451,0.696030,-1.582880,-1.970906,-2.797395,-4.170106,-2.897393,...,-3.671383,-2.043819,-2.117368,-2.359486,-2.096747,-1.316318,-1.873511,-5.601148,-1.699904,-2.786461
ENSG00000179914,-3.569415,2.119767,-4.083801,-5.261526,-5.056427,-3.801708,-2.079691,-3.225063,-0.174631,-9.965784,...,-1.079852,-2.590863,-2.664269,-6.267381,-1.360455,-1.269543,1.355935,1.190012,-1.504873,-5.427190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000203859,-2.559716,-2.424260,-5.743966,-3.790421,-0.057156,-5.137312,-2.676033,-4.527592,-9.965784,-4.219806,...,-6.387894,-1.101986,-9.965784,-2.376231,1.592210,3.395194,-2.303537,-3.861291,-3.788814,-4.773279
ENSG00000164270,-9.965784,-7.681085,-9.569261,-9.965784,-9.511735,-9.965784,-8.773356,-9.965784,-9.563134,-9.965784,...,-9.965784,-9.965784,-7.821315,-9.422141,-9.965784,-9.965784,-9.398801,-9.205281,-9.252905,-9.965784
ENSG00000185823,-0.891385,-2.015031,-2.334598,-4.369244,-6.382877,-9.965784,-0.802130,-0.124686,-9.965784,-6.058232,...,-9.965784,-2.631388,-9.965784,-4.175109,0.006519,-0.308931,-3.306644,-3.128442,0.756916,-9.965784


In [14]:
df_data_mod = df_data.T

In [15]:
df_data_mod

ID,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,ENSG00000136944,ENSG00000182870,...,ENSG00000077327,ENSG00000196131,ENSG00000136110,ENSG00000171053,ENSG00000150361,ENSG00000203859,ENSG00000164270,ENSG00000185823,ENSG00000142789,ENSG00000183715
TCGA-04-1331,-6.366282,-0.442050,0.566665,-2.268812,-3.569415,1.882780,-9.965784,-0.441449,-0.677656,-5.400592,...,-1.053987,-9.965784,-6.816989,-3.622838,-7.966778,-2.559716,-9.965784,-0.891385,-6.136564,-9.791117
TCGA-04-1332,-4.488628,-3.216314,-2.666762,-1.536532,2.119767,1.832807,-9.965784,-1.130959,1.164030,-4.032188,...,-7.210435,-4.783838,-5.536844,-2.511355,-9.965784,-2.424260,-7.681085,-2.015031,-9.965784,-8.039971
TCGA-04-1347,-7.601309,-3.024444,-8.117399,-2.476620,-4.083801,5.096513,-5.581598,-3.766292,-8.563616,-8.580961,...,-2.072122,-9.965784,-4.441578,-9.965784,-9.198811,-5.743966,-9.569261,-2.334598,-7.044777,-9.876247
TCGA-04-1362,-4.173178,-1.696708,-4.539869,-3.195451,-5.261526,-1.795652,-4.030746,0.032396,-2.371091,-5.497429,...,-2.027688,-9.965784,-8.392890,-9.965784,-9.965784,-3.790421,-9.965784,-4.369244,-9.965784,-9.790471
TCGA-04-1364,-2.754576,6.288751,-8.817501,0.696030,-5.056427,4.264846,-9.965784,-0.344290,-6.559042,-6.982357,...,-0.762673,-1.494008,-3.203281,-5.361784,-9.696609,-0.057156,-9.511735,-6.382877,-9.965784,-9.673470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-2098,-3.424341,5.523327,-3.943922,-1.316318,-1.269543,2.324090,-1.967411,6.121391,-0.534620,-5.726127,...,0.048487,-3.669741,-2.927971,-2.475176,-9.040579,3.395194,-9.965784,-0.308931,-3.552707,-9.965784
TCGA-61-2109,-6.197002,0.155893,-4.107782,-1.873511,1.355935,3.306055,-4.999621,-0.662207,-4.365621,-4.251542,...,-8.964987,-9.965784,-5.516037,-9.965784,-9.624395,-2.303537,-9.398801,-3.306644,-5.572844,-7.510953
TCGA-61-2110,-6.249893,-1.092131,-7.348430,-5.601148,1.190012,-3.328315,-9.965784,0.209943,-2.838656,-0.623255,...,-9.965784,-9.965784,-8.311607,-3.907536,-9.495995,-3.861291,-9.205281,-3.128442,-9.965784,-8.363088
TCGA-61-2111,2.055320,4.107675,-8.332655,-1.699904,-1.504873,-1.043990,1.917116,-2.633256,-5.097214,-5.531636,...,-5.773895,-9.965784,-9.965784,-9.965784,-9.528129,-3.788814,-9.252905,0.756916,-4.207623,-9.965784


In [16]:
#df_label_mi = pd.read_csv('./Platin_Data/miRNA_patient.csv', sep = '\t', index_col = 0)

In [17]:
df_data_mi = pd.read_csv('./Platin_Data/Semi_Final/MIR_rpm_100_log.csv', sep = '\t', index_col = 0)

In [18]:
df_data_mi = df_data_mi.T

In [19]:
df_data_lnc = pd.read_csv('./Platin_Data/Semi_Final/LNC_tpm_50_log.csv', sep = '\t', index_col = 0)

In [20]:
df_data_lnc

,TCGA-04-1331,TCGA-04-1332,TCGA-04-1347,TCGA-04-1362,TCGA-04-1364,TCGA-04-1365,TCGA-04-1514,TCGA-04-1530,TCGA-04-1536,TCGA-04-1542,...,TCGA-61-2000,TCGA-61-2008,TCGA-61-2009,TCGA-61-2092,TCGA-61-2097,TCGA-61-2098,TCGA-61-2109,TCGA-61-2110,TCGA-61-2111,TCGA-61-2113
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000233048,-7.207620,-9.965784,-9.965784,-9.965784,-3.797716,-8.781751,-8.888570,-9.965784,-9.965784,-8.813903,...,-9.965784,-8.850686,-9.965784,-9.965784,-9.965784,-7.098053,-7.110620,-9.965784,-7.035556,-9.965784
ENSG00000083622,-9.965784,-4.786606,-9.965784,-6.722537,-9.965784,-8.115396,-9.965784,-5.869315,-9.965784,-9.965784,...,-9.965784,-9.965784,-9.965784,-9.965784,-9.965784,-9.965784,-6.850643,-7.259768,-4.471057,-5.553862
ENSG00000269994,-8.325863,-5.686705,-6.729526,-6.184552,-9.965784,-7.051607,-8.020065,-5.808990,-7.563521,-9.965784,...,-5.907724,-7.967025,-8.022048,-7.949062,-9.965784,-6.406115,-6.557792,-6.609472,-1.229587,-6.343955
ENSG00000258752,-0.500069,-3.869310,-6.173850,-5.772887,0.101508,-5.179247,0.664458,-3.996567,-6.684953,-9.965784,...,-9.965784,-3.024313,-5.766190,-1.121523,-5.421963,-0.779119,-7.055175,-4.529617,-5.771347,-5.380351
ENSG00000249790,-6.567569,-3.572952,-4.681104,-6.562452,-4.460856,-5.989466,-3.936696,-9.965784,-9.965784,-9.965784,...,-4.378901,-5.157279,-3.679993,-3.843015,-2.160467,-2.891021,-6.016198,-9.965784,-9.965784,-4.265393
ENSG00000261399,0.844999,1.141537,2.071746,-2.230827,0.435589,2.165586,1.579985,2.181894,-9.965784,-9.965784,...,-2.930124,-0.417775,-1.810424,1.675046,-0.282066,-1.887774,3.213477,0.054455,1.224064,0.345725
ENSG00000232721,-9.965784,-1.077274,-5.519466,-1.248935,-0.604508,-9.965784,0.173711,-9.965784,-3.544945,-4.964940,...,-5.222931,-0.827823,-0.395389,1.116185,0.495997,-3.626908,-2.969993,-3.119024,-2.243889,-4.136419
ENSG00000243479,-6.179796,-5.425791,-9.965784,-2.028888,-5.976203,-6.522107,-4.816674,-4.583403,-2.676088,-3.121561,...,1.380614,-2.776810,-9.965784,2.101699,3.953488,-9.965784,-2.223574,-5.111212,-0.471729,-3.051074
ENSG00000248554,-1.826764,-2.248224,-3.752458,-3.118508,-1.830784,0.104746,-1.397632,-2.843421,-4.012739,-2.558986,...,-3.102304,-4.370490,-3.776527,-2.880645,-2.670585,-0.317675,-0.868077,-3.620514,-0.892318,-1.756998


In [21]:
df_data_lnc = df_data_lnc.T

In [22]:
#df_mi = pd.concat([df_label_mi,df_data_mi], axis = 1)

In [23]:
df = pd.concat([df_label,df_data_mod,df_data_mi,df_data_lnc], axis = 1)

In [57]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.2)

In [58]:
train

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,...,ENSG00000231621,ENSG00000225472,ENSG00000231764,ENSG00000241388,ENSG00000233536,ENSG00000234692,ENSG00000224717,ENSG00000262117,ENSG00000248646,ENSG00000258170
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-25-1315,Resistant,1.0,-4.686983,0.960027,-7.704844,-0.728522,-1.069286,1.390239,-5.708668,0.628004,...,5.178812,-6.200513,-2.085854,-7.030567,-9.965784,-0.625400,-0.932508,-1.614775,-9.965784,-9.965784
TCGA-24-0979,Sensitive,0.0,-5.813098,-9.965784,-8.214181,0.696332,-4.119434,0.170307,-5.958588,2.285056,...,4.476964,-9.965784,-9.024147,-8.052592,-9.965784,-2.166303,-6.218003,-5.713393,-7.978815,-6.137611
TCGA-61-2008,Sensitive,0.0,-6.816355,-1.936841,-7.028077,-2.043819,-2.590863,-2.956409,-9.965784,1.944798,...,-0.844354,-9.965784,-6.255257,-8.119791,-9.965784,-4.048360,-1.680290,-9.965784,-9.965784,-5.674618
TCGA-61-2109,Sensitive,0.0,-6.197002,0.155893,-4.107782,-1.873511,1.355935,3.306055,-4.999621,-0.662207,...,1.443201,-6.440047,-5.218195,-8.049145,2.099915,-2.964156,-3.310530,-9.965784,-9.965784,-9.965784
TCGA-25-1626,Resistant,1.0,-6.062823,-3.492144,-7.158498,-1.574098,-5.328884,2.505427,-9.965784,1.841454,...,0.950276,-5.761374,-2.844205,-6.814008,-9.965784,-9.965784,-3.647855,-1.859318,-9.965784,-4.287891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-31-1950,Sensitive,0.0,-3.515936,-1.504317,-2.365897,-4.508600,1.745130,0.777006,-1.062668,-3.695113,...,-9.965784,-5.578493,-6.175614,-6.963735,-3.011187,-9.965784,-4.942857,-5.942840,-9.965784,-2.222087
TCGA-13-0905,Sensitive,0.0,-4.464729,1.093583,0.827830,-4.394041,-0.444499,3.603272,-5.029925,-6.108373,...,1.520381,-1.889686,-7.282487,-8.072101,-4.675745,-2.414103,-1.766353,-7.177194,-9.965784,-3.925545
TCGA-24-1434,Resistant,1.0,-7.535915,-3.933301,-6.844471,-4.688200,-5.196260,1.075179,-9.965784,-3.686641,...,-9.965784,-7.747252,-1.302654,-7.203258,-9.965784,-2.904797,-3.609945,-6.734171,-8.942039,-6.077553


In [59]:
train.to_csv("train_platin_total_lnc_400_log.csv", mode='w')

In [60]:
val

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,...,ENSG00000231621,ENSG00000225472,ENSG00000231764,ENSG00000241388,ENSG00000233536,ENSG00000234692,ENSG00000224717,ENSG00000262117,ENSG00000248646,ENSG00000258170
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-23-1027,Resistant,1.0,-6.135284,-3.349095,-3.863475,-3.246019,-7.532949,-0.258138,-6.434488,-4.244782,...,1.625110,-9.965784,-6.169082,-7.999968,-9.965784,-9.965784,-5.762840,-6.195148,-9.965784,-9.965784
TCGA-36-1580,Resistant,1.0,-5.473947,-1.396137,-6.964441,-1.891926,-6.705484,1.140224,-5.532685,-1.031337,...,1.616526,-6.412237,-5.747777,-7.228279,-9.965784,-4.497043,-1.194321,-5.840806,-9.965784,-6.104939
TCGA-29-1769,Sensitive,0.0,-5.476657,-1.177117,-6.966923,-3.928615,-2.467021,0.327985,-9.965784,-5.223210,...,0.134540,-5.597427,-8.153995,-7.711185,-9.965784,-2.204239,-7.232189,-5.843480,-9.965784,-5.158198
TCGA-23-1122,Sensitive,0.0,-4.735521,-0.658595,-2.085734,-4.534910,-0.325670,4.737879,-9.965784,0.939584,...,-0.660026,-5.296299,-1.508721,-6.987275,-2.043833,-2.336167,-3.495986,-5.968097,-8.458205,-4.200967
TCGA-24-1474,Sensitive,0.0,-7.167001,-1.189044,-7.722494,-3.270644,-4.047874,-0.493115,-5.080759,-3.250298,...,-2.050279,-1.962848,-6.970814,-7.324875,-4.726960,-4.035504,-2.662685,-9.965784,-9.965784,-4.699211
TCGA-10-0927,Sensitive,0.0,-4.488306,-2.955190,-6.264053,-4.698829,1.430639,0.653075,-9.965784,0.464706,...,2.036249,-7.755812,-9.965784,-9.965784,-9.965784,-9.965784,0.170059,-9.965784,-9.965784,-9.965784
TCGA-61-1914,Sensitive,0.0,-2.093100,-0.684666,-5.571586,-1.231573,-3.907345,1.061018,-2.296048,-3.235534,...,4.319977,-7.669277,-5.358908,-4.248088,-9.965784,-0.152473,-3.618159,-9.965784,-9.965784,-9.965784
TCGA-29-1703,Resistant,1.0,-0.601242,-0.924891,-4.594849,-2.106812,-1.743499,4.825219,-9.965784,-1.882055,...,4.161422,-6.514948,-1.338677,-9.965784,-9.965784,-1.468727,-4.158751,-6.332438,-8.700067,-4.480549
TCGA-04-1365,Sensitive,0.0,-6.551323,-0.008033,-0.929409,-1.582880,-3.801708,3.537387,-2.837690,-5.286824,...,-0.526685,-5.660294,-2.012460,-7.574461,-9.965784,-1.772033,-0.801980,-6.230265,-8.634323,-1.994469


In [61]:
val.to_csv("val_platin_total_lnc_400_log.csv", mode='w')

In [62]:
#train = pd.read_csv("train_platin_total_lnc_400.csv")
#val = pd.read_csv("val_platin_total_lnc_400.csv")

In [2]:
train = pd.read_csv("train_platin_total_lnc_400_log.csv")
val = pd.read_csv("val_platin_total_lnc_400_log.csv")

In [3]:
val_label = val['PATIENT_ID']

In [4]:
trn_X_pd = train.drop(["PATIENT_ID","PLATINUM_STATUS","label"],axis=1).values
trn_y_pd = train.label.values

print("Information of Training Data")
print("Samples : {}".format(trn_X_pd.shape[0]))
print("Features : {}".format(trn_X_pd.shape[1]))

Information of Training Data
Samples : 168
Features : 400


In [5]:
val_X_pd = val.drop(["PATIENT_ID","PLATINUM_STATUS","label"],axis=1).values
val_y_pd = val.label.values

print("Information of Training Data")
print("Samples : {}".format(val_X_pd.shape[0]))
print("Features : {}".format(val_X_pd.shape[1]))

Information of Training Data
Samples : 42
Features : 400


In [6]:
item = []
item_2 = []

print( (val_y_pd == 1.0).sum())
print( (val_y_pd == 0.0).sum())

print( (trn_y_pd == 1.0).sum())
print( (trn_y_pd == 0.0).sum())

13
29
48
120


In [7]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5 # Number of K-fold Splits

splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(trn_X_pd, trn_y_pd))
splits[:5]

[(array([  1,   4,   6,   7,  10,  11,  12,  13,  14,  15,  16,  17,  18,
          19,  20,  22,  23,  27,  28,  29,  30,  31,  33,  34,  36,  37,
          38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  49,  50,  52,
          53,  54,  55,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
          68,  70,  72,  73,  74,  76,  78,  79,  80,  81,  82,  83,  84,
          87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  98,  99, 100,
         101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114,
         115, 116, 117, 118, 119, 120, 122, 124, 126, 127, 128, 129, 130,
         131, 132, 134, 135, 137, 138, 139, 141, 142, 143, 144, 145, 146,
         147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
         162, 163, 164, 166]),
  array([  0,   2,   3,   5,   8,   9,  21,  24,  25,  26,  32,  35,  48,
          51,  56,  57,  69,  71,  75,  77,  85,  86,  97, 109, 121, 123,
         125, 133, 136, 140, 160, 161, 165, 167])),
 (array([  0,   1,   2,   3, 

In [8]:
for i, (train_idx, valid_idx) in enumerate(splits):

    print( (trn_y_pd[valid_idx.astype(int)] == 1.0).sum())
    print( (trn_y_pd[valid_idx.astype(int)] == 0.0).sum())
    
    #print( (trn_y_pd[train_idx.astype(int)] == 1.0).sum())
    #print( (trn_y_pd[train_idx.astype(int)] == 0.0).sum())

10
24
10
24
10
24
9
24
9
24


## Dataset

In [9]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [10]:
batch_size = 5

## Model

In [11]:
class DNN_seq_1(nn.Module):
    def __init__(self):
        super(DNN_seq_1, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(200, 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(200,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [112]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(100, 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(200, 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            torch.nn.Linear(200, 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(200, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [13]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_2(nn.Module):
    def __init__(self):
        super(DNN_seq_2, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(100, 50, bias=True),
            torch.nn.BatchNorm1d(50),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(50, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(100, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


## Scheduler control

In [14]:
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [15]:
from torch.optim.optimizer import Optimizer

import time # ??

from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [16]:
print(trn_X_pd.shape[0])
          
train_preds = np.zeros(trn_X_pd.shape[0])
## Addiction
train_y_sort = np.zeros(trn_X_pd.shape[0])

test_preds = np.zeros(val_X_pd.shape[0])


train_target = train.label

168


In [468]:
avg_losses_f = []
avg_val_losses_f = []

for i, (train_idx, valid_idx) in enumerate(splits):
    
    ## ???
    x_train_fold = torch.tensor(trn_X_pd[train_idx.astype(int)], dtype=torch.float)   # use_cuse?
    y_train_fold = torch.tensor(trn_y_pd[train_idx.astype(int), np.newaxis], dtype=torch.float)    

    x_val_fold = torch.tensor(trn_X_pd[valid_idx.astype(int)], dtype=torch.float)
    y_val_fold = torch.tensor(trn_y_pd[valid_idx.astype(int), np.newaxis], dtype=torch.float)  
    
################################################################################    
    trn_X = torch.from_numpy(trn_X_pd[train_idx.astype(int)].astype(float))
    trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
    
    # Train
    trn = Dataset(trn_X, trn_y)
    trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)  # True or False
    trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False, drop_last = False)  # True or False
    
    # Valid
    valid_X = torch.from_numpy(trn_X_pd[valid_idx.astype(int)].astype(float))
    valid_y = torch.from_numpy(trn_y_pd[valid_idx.astype(int)].astype(float))
    
    valid = Dataset(valid_X, valid_y)
    valid_loader = data_utils.DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last = False) # True or False
    
    # Test
    val_X = torch.from_numpy(val_X_pd.astype(float))
    val_y = torch.from_numpy(val_y_pd.astype(float))
    val = Dataset(val_X, val_y)
    test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)
   ################################################################################    
    print(i)
    
    ## Loss function
    criterion = nn.BCELoss()
    learning_rate = 1e-3
    
    model = DNN_seq_2()
    ## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
    model = DNN_seq_2()
    use_cuda = torch.cuda.is_available()
    num_epochs = 50
    
    if use_cuda:
        model = model.cuda()
    
    ##############################################################################
    step_size = 2000
    base_lr, max_lr = 0.001, 0.01  
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
    scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    
    
    ##############################################################################
    
    print(f'Fold {i+1}')
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.
        #correct = 0.   # Accuracy
        
        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            #optimizer.zero_grad()
            trn_pred = model(trn_X)

            
            if scheduler:
                #print('cycle_LR')
                scheduler.batch_step()
            #print(trn_pred.squeeze())
            #print(trn_y)
            trn_loss = criterion(trn_pred.squeeze(), trn_y)
            optimizer.zero_grad()
            trn_loss.backward()
            optimizer.step()
        
            avg_loss += trn_loss.item()/len(trn_loader)
        
            #predicted = torch.max(trn_pred.data, 1)[1] 
            #correct += (predicted == trn_y).sum()
        model.eval()
        
        valid_preds_fold = np.zeros((valid_X.size(0)))
        test_preds_fold = np.zeros(val_X_pd.shape[0])  # Test
        
        avg_val_loss = 0.
        
        with torch.no_grad():
            for batch_idx, val in enumerate(valid_loader):
                val_X, val_y = val['X'], val['y']
                if use_cuda:
                    val_X, val_y = val_X.cuda(), val_y.cuda()
                val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
                #optimizer.zero_grad()
                val_pred = model(val_X).detach()

            
                val_loss = criterion(val_pred.squeeze(), val_y)
        
                avg_val_loss += val_loss.item()/len(valid_loader)
            
            
                #val_pred = torch.max(val_pred, 1)[1]
                val_pred = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
                #print(val_pred.cpu().numpy())
                #print(val_pred.cpu().numpy()[:,0])
            
                valid_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (val_pred.cpu().numpy())    # modified [:,0]
            
            #valid_preds_fold_2 = 
                # Loss function chage -> plus Sigmoid
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(epoch + 1, num_epochs, avg_loss, avg_val_loss, elapsed_time))            

# Test        
    avg_losses_f.append(avg_loss)
    avg_val_losses_f.append(avg_val_loss) 
    
    for batch_idx, test in enumerate(test_loader):
        test_X, test_y = test['X'], test['y']
        if use_cuda:
            test_X, test_y = test_X.cuda(), test_y.cuda()
        test_X, test_y = Variable(test_X).float(), Variable(test_y).float()        
        test_pred = model(test_X).detach()
        
        
        test_pred = (test_pred > 0.5).flatten().type(torch.ByteTensor)
        
        #print(test_pred)
        #print(test_pred.cpu().numpy())
        test_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (test_pred.cpu().numpy())   # modified [:,0]
        
        
    train_preds[valid_idx.astype(int)] = valid_preds_fold
    print(valid_preds_fold)
    print(trn_y_pd[valid_idx.astype(int)])
    train_y_sort[valid_idx.astype(int)] = trn_y_pd[valid_idx.astype(int)]
    test_preds += test_preds_fold / len(splits)

#predict_ = pd.DataFrame(predict)

#predict_.iloc[:,0]

#label = pd.DataFrame(val_y_pd)

#test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)    
    
print(trn_y_pd)
print(train_preds)

auc  =  round(roc_auc_score(train_y_sort,train_preds.astype(int)),4)


cnf = confusion_matrix(train_y_sort, train_preds, labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

print('AUC: ', auc)


#fpr, trp, _  =  roc_auc_score(train_y_sort,train_preds.astype(int))

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

#print(type(fpr))
#print(fpr)

#auc = auc(fpr, trp)
#print('AUC: ', auc(fpr, trp))
print('\n')
print('All \t loss={:.4f} \t val_loss={:.4f} \t auc={:.4f}'.format(np.average(avg_losses_f),np.average(avg_val_losses_f),auc))        
        

0
Fold 1
Epoch 1/50 	 loss=0.7050 	 val_loss=0.6449 	 time=0.21s
Epoch 2/50 	 loss=0.5987 	 val_loss=0.6233 	 time=0.18s
Epoch 3/50 	 loss=0.5652 	 val_loss=0.6145 	 time=0.17s
Epoch 4/50 	 loss=0.6352 	 val_loss=0.6055 	 time=0.17s
Epoch 5/50 	 loss=0.5130 	 val_loss=0.5887 	 time=0.18s
Epoch 6/50 	 loss=0.5710 	 val_loss=0.5964 	 time=0.17s
Epoch 7/50 	 loss=0.5605 	 val_loss=0.5501 	 time=0.19s
Epoch 8/50 	 loss=0.5191 	 val_loss=0.5774 	 time=0.18s
Epoch 9/50 	 loss=0.5207 	 val_loss=0.5819 	 time=0.18s
Epoch 10/50 	 loss=0.4991 	 val_loss=0.6172 	 time=0.18s
Epoch 11/50 	 loss=0.5348 	 val_loss=0.6020 	 time=0.22s
Epoch 12/50 	 loss=0.4355 	 val_loss=0.5398 	 time=0.18s


KeyboardInterrupt: 

# Valid

In [80]:
model.eval()
predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.
    for j, val in enumerate(valid_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred > 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Train', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(valid_loader)*batch_size)))
print('*'*46)
print("Train accuracy:{:.3f}".format(float(correct_val) / (len(valid_loader)*batch_size)))

tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([1, 1, 0, 0, 1], dtype=torch.uint8)
tensor([1., 0., 0., 0., 0.])
tensor([0, 1, 0, 0, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 1, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 1., 1., 1.])
tensor([1, 1, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([1, 1, 0], dtype=torch.uint8)
tensor([1., 1., 0.])
******************** Train ********************
Loss: 0.18184316158294678, Accuracy: 0.7714285850524902 %
**********************************************
Train accuracy:0.771


In [81]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(trn_y_pd[valid_idx.astype(int)], predict_.iloc[:,0])

,0
0,
0.426939,0.0
0.268852,0.0
0.040547,0.0
0.003152,0.0
0.358020,0.0
0.750645,1.0
0.537834,0.0
0.376175,0.0
0.449809,0.0


In [82]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd[valid_idx.astype(int)])

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])
auc  =  round(roc_auc_score(train_y_sort,train_preds.astype(int)),4)
#print(type(fpr))
#print(fpr)
print('AUC: ', auc)

[[ 8  1]
 [ 5 19]]
Accuracy :  0.8181818181818182
Sensitivity :  0.8888888888888888
Specificity :  0.7916666666666666
AUC:  0.7396


In [83]:
len(x_train_fold)

135

# Test

In [249]:
model = DNN_seq_2()

In [259]:
#model = TheModelClass(*args, **kwargs)                                                                                                     
#optimizer = TheOptimizerClass(*args, **kwargs)                                                                                             
checkpoint = torch.load("./platin_model_save/platin_model_400_model_2_100.pth")                                                                                  
model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
#num_epochs = checkpoint['epoch']                                                                                                           
loss = checkpoint['loss']                                                                                                                   
model.eval()  

DNN_seq_2(
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
  (classifier3): Sequential(
    (0): Linear(in_features=50, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=100, out_features=250, bias=True)
    (5): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=250, out

In [84]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 1, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 0., 0.])
tensor([1, 0, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 1, 0, 0, 1], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 1, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([1, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.30355265736579895, Accuracy: 0.6888889074325562 %
**********************************************
Val accuracy:0.689


In [85]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.709781,1.0
0.520516,1.0
0.406429,0.0
0.002304,0.0
0.033847,0.0
0.510784,0.0
0.047692,0.0
0.717254,1.0
0.366883,0.0


In [86]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[ 9  4]
 [ 7 22]]
Accuracy :  0.7380952380952381
Sensitivity :  0.6923076923076923
Specificity :  0.7586206896551724
<class 'numpy.ndarray'>
AUC:  0.7254641909814323


In [33]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_400_model_2_100__1_test.pth")      

# Train Test (Except Valid)

In [137]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[0.7690],
        [0.0011],
        [0.0015],
        [0.0312],
        [0.0142]])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[1.1163e-01],
        [2.5054e-04],
        [6.5338e-02],
        [3.7494e-01],
        [1.0833e-01]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[0.4652],
        [0.0397],
        [0.6512],
        [0.1571],
        [0.5317]])
tensor([0, 0, 1, 0, 1], dtype=torch.uint8)
tensor([[0.0034],
        [0.0203],
        [0.0044],
        [0.7189],
        [0.6362]])
tensor([0, 0, 0, 1, 1], dtype=torch.uint8)
tensor([[2.8724e-04],
        [8.3109e-02],
        [1.2839e-02],
        [6.5839e-01],
        [7.0855e-01]])
tensor([0, 0, 0, 1, 1], dtype=torch.uint8)
tensor([[0.1849],
        [0.1659],
        [0.0229],
        [0.0023],
        [0.7395]])
tensor([0, 0, 0, 0, 1], dtype=torch.uint8)
tensor([[0.0014],
        [0.0047],
        [0.0008],
        [0.0753],
        [0.7521]])
tensor([0, 0, 0, 0, 1], dtype=torch.uint8)
tensor([[0.2553],
    

In [138]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd[train_idx.astype(int)], predict_.iloc[:,0])

,0
0,
0.769005,1.0
0.001134,0.0
0.001475,0.0
0.031231,0.0
0.014199,0.0
...,...
0.663252,1.0
0.835189,1.0
0.694276,1.0


In [139]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd[train_idx.astype(int)])

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[38  1]
 [ 0 96]]
Accuracy :  0.9925925925925926
Sensitivity :  0.9743589743589743
Specificity :  1.0
AUC:  0.9871794871794872


# .........................................................................

# Real Train stage

# .........................................................................

# Train (Total Data)

In [115]:
class DNN_seq_1(nn.Module):
    def __init__(self):
        super(DNN_seq_1, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.25),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(100, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.25),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [136]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [137]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [138]:
len(test_loader)

9

In [118]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3

model = DNN_seq_1()    

## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
model = DNN_seq_1()
use_cuda = torch.cuda.is_available()
num_epochs = 70
    
if use_cuda:
    model = model.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [140]:
###########################
model_load = 0
###########################

In [141]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.40207138657569885, Accuracy: 0.6606060862541199 % 	 time=0.16s
******************** Train ********************
Loss: 0.27406561374664307, Accuracy: 0.7939394116401672 % 	 time=0.12s
******************** Train ********************
Loss: 0.2302365005016327, Accuracy: 0.8545454740524292 % 	 time=0.14s
******************** Train ********************
Loss: 0.1906513273715973, Accuracy: 0.8969696760177612 % 	 time=0.13s
******************** Train ********************
Loss: 0.10916025936603546, Accuracy: 0.9090909361839294 % 	 time=0.15s
******************** Train ********************
Loss: 0.04991741105914116, Accuracy: 0.939393937587738 % 	 time=0.13s
******************** Train ********************
Loss: 0.04607832804322243, Accuracy: 0.9515151381492615 % 	 time=0.16s
******************** Train ********************
Loss: 0.02667769230902195, Accuracy: 0.9272727370262146 % 	 time=0.14s
******************** Train ********************
Los

In [308]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[9.5983e-01],
        [2.9158e-05],
        [7.6539e-03],
        [3.2472e-06],
        [9.9180e-01]])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([[1.4340e-04],
        [1.0315e-05],
        [2.7879e-06],
        [9.8068e-01],
        [3.5452e-04]])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([[4.7349e-06],
        [8.0224e-03],
        [5.9592e-05],
        [1.7694e-10],
        [9.9776e-04]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[3.3224e-06],
        [1.7900e-10],
        [7.3089e-07],
        [8.9265e-06],
        [1.4304e-03]])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[4.6982e-09],
        [3.3153e-06],
        [9.8132e-01],
        [3.7413e-02],
        [8.2993e-12]])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([[9.9242e-01],
        [1.0599e-06],
        [1.4360e-05],
        [2.2877e-05],
        [9.8047e-01]])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([[9.8905e-01],
        [5.1682e-06],
        [3.6797e-04],
        [7

In [309]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd, predict_.iloc[:,0])

,0
0,
9.598317e-01,1.0
2.915792e-05,0.0
7.653852e-03,0.0
3.247193e-06,0.0
9.917979e-01,1.0
...,...
8.439797e-05,0.0
6.015256e-10,0.0
9.839814e-01,1.0


In [310]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 48   0]
 [  0 120]]
Accuracy :  1.0
Sensitivity :  1.0
Specificity :  1.0
AUC:  1.0


# Final Test

In [459]:
model_load = 1

In [460]:
if (model_load == 1):
    model = DNN_seq_1()
    #model = TheModelClass(*args, **kwargs)                                                                                                     
    #optimizer = TheOptimizerClass(*args, **kwargs)                                                                                             
    checkpoint = torch.load("./platin_model_save/platin_model_250_50_100_model_1_100_250_Semi_Final_Voting.pth")                                                                                  
    model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
    #num_epochs = checkpoint['epoch']                                                                                                           
    loss = checkpoint['loss']                                                                                                                   
    model.eval() 
    print('model_load check')

model_load check


In [461]:
len(test_loader)

9

In [462]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 1, 1, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 0., 0.])
tensor([0, 0, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 1, 0, 1, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 1, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 1, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([1, 0, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.0005658600712195039, Accuracy: 0.7777777910232544 %
**********************************************
Val accuracy:0.778


In [463]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
9.534943e-01,1.0
6.911207e-01,1.0
6.269382e-01,0.0
1.569815e-09,0.0
3.728391e-12,0.0
1.387491e-02,0.0
6.077649e-13,0.0
9.466208e-01,1.0
1.156189e-02,0.0


In [520]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p_1 = pd.concat([predict_.iloc[:,0], label, val_label, val_label], axis = 1)

#print(test_p)

test_p_1.columns = ['predict','label','index_ID', 'ID']
test_p_1.loc[test_p_1['predict'] >= 0.5, 'predicted_prob'] = 1
test_p_1.loc[test_p_1['predict'] < 0.5, 'predicted_prob'] = 0


test_p.columns = ['predict','label']

test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[12  1]
 [ 6 23]]
Accuracy :  0.8333333333333334
Sensitivity :  0.9230769230769231
Specificity :  0.7931034482758621
<class 'numpy.ndarray'>
AUC:  0.8580901856763927


In [39]:
cnf = confusion_matrix(test_p_1['label'], test_p_1['predicted_prob'], labels = [1,0])

print(test_p_1.set_index('index_ID'))

test_p_1temp = test_p_1.set_index('index_ID')


print(cnf)
###print(pd.DataFrame(test_p_2))
is_index = test_p_1temp['predicted_prob'] == 1.0

#print(is_index)

#print(test_p_2.loc[test_p_2['predict'] < 0.5, 'predicted_prob'])
test_1 = test_p_1temp[test_p_1temp['predicted_prob'] == 1.0]
#test_p_temp = test_p_2.drop['ID']

#print(test_p_2)
print(len(test_1))
test_1
#mask = test_p_2[test_p_2['label'].isin(0)]

#test_p_2[~mask.astype(int)].head()

#mask = mock_data['country'].isin(['Afghanistan', 'Nigeria']) mock_data[~mask].head()

#출처: https://note.espriter.net/1325 [espriter]

                   predict  label            ID  predicted_prob
index_ID                                                       
TCGA-23-1027  9.534943e-01    1.0  TCGA-23-1027             1.0
TCGA-36-1580  6.911207e-01    1.0  TCGA-36-1580             1.0
TCGA-29-1769  6.269382e-01    0.0  TCGA-29-1769             1.0
TCGA-23-1122  1.569815e-09    0.0  TCGA-23-1122             0.0
TCGA-24-1474  3.728391e-12    0.0  TCGA-24-1474             0.0
TCGA-10-0927  1.387491e-02    0.0  TCGA-10-0927             0.0
TCGA-61-1914  6.077649e-13    0.0  TCGA-61-1914             0.0
TCGA-29-1703  9.466208e-01    1.0  TCGA-29-1703             1.0
TCGA-04-1365  1.156189e-02    0.0  TCGA-04-1365             0.0
TCGA-29-1696  9.542856e-01    1.0  TCGA-29-1696             1.0
TCGA-13-1483  8.174533e-01    1.0  TCGA-13-1483             1.0
TCGA-61-1738  5.389222e-01    1.0  TCGA-61-1738             1.0
TCGA-61-1911  1.452639e-04    0.0  TCGA-61-1911             0.0
TCGA-25-2393  5.486322e-01    1.0  TCGA-

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-23-1027,0.953494,1.0,TCGA-23-1027,1.0
TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0
TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0
TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0
TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0
TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0
TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0
TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0
TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0


# Model Save

# require to check~~

In [178]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_250_50_100_model_1_100_250_Semi_Final_Voting___test.pth")     

In [525]:
test_merge = pd.merge(test_1, test_2, left_on = ['ID'], right_index = True)

test_merge

#test_merge_2 = pd.merge(test_merge, test_3, left_on = ['ID'], right_index = True)

#test_merge_2

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y
index_ID,,,,,,,,,
TCGA-36-1580,TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0,0.083512,1.0,TCGA-36-1580,0.0
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0
TCGA-61-1738,TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0,0.151342,1.0,TCGA-61-1738,0.0
TCGA-25-2393,TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0,0.222550,1.0,TCGA-25-2393,0.0
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0


In [586]:
test_merge.loc[(test_merge['predict_y'] < 0.08) & (test_merge['predict_x'] - test_merge['predict_y'] > 0.5) ]

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y
index_ID,,,,,,,,,
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0


In [389]:
test_merge['mean_predict'] = (test_merge['predict_x'] * 0.8 + test_merge['predict_y'] * 0.2)

test_merge

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y,mean_predict
index_ID,,,,,,,,,,
TCGA-36-1580,TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0,0.083512,1.0,TCGA-36-1580,0.0,0.569599
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0,0.510004
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0,0.857282
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0,0.851092
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0,0.697256
TCGA-61-1738,TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0,0.151342,1.0,TCGA-61-1738,0.0,0.461406
TCGA-25-2393,TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0,0.222550,1.0,TCGA-25-2393,0.0,0.483416
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0,0.702049
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0,0.581938


In [540]:
mod_index = test_merge.loc[(test_merge['predict_y'] < 0.08) ]

mod_index

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y
index_ID,,,,,,,,,
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0


In [391]:
test_merge.loc[(test_merge['predict_x'] > 0.8) ]

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y,mean_predict
index_ID,,,,,,,,,,
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0,0.857282
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0,0.851092
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0,0.697256
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0,0.702049
TCGA-13-0893,TCGA-13-0893,0.863482,1.0,TCGA-13-0893,1.0,0.409376,1.0,TCGA-13-0893,0.0,0.772660
TCGA-13-1403,TCGA-13-1403,0.902720,0.0,TCGA-13-1403,1.0,0.200409,0.0,TCGA-13-1403,0.0,0.762258
TCGA-09-2056,TCGA-09-2056,0.941217,0.0,TCGA-09-2056,1.0,0.310341,0.0,TCGA-09-2056,0.0,0.815042


In [528]:
test_temp_df = test_merge.loc[(test_merge['predict_y'] >= 0.2) | (test_merge['predict_x'] <= 0.8) ]
test_temp_df

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y
index_ID,,,,,,,,,
TCGA-36-1580,TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0,0.083512,1.0,TCGA-36-1580,0.0
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0
TCGA-61-1738,TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0,0.151342,1.0,TCGA-61-1738,0.0
TCGA-25-2393,TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0,0.222550,1.0,TCGA-25-2393,0.0
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0


In [533]:
test_temp_merge = pd.merge(test_temp_df, test_3, left_on = ['ID'], right_index = True)

test_temp_merge

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y,predict_add,label_add,index_ID,predicted_prob_add
index_ID,,,,,,,,,,,,,
TCGA-36-1580,TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0,0.083512,1.0,TCGA-36-1580,0.0,0.377572,1.0,TCGA-36-1580,0.0
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0,0.105563,0.0,TCGA-29-1769,0.0
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0,0.937590,1.0,TCGA-29-1703,1.0
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0,0.975946,1.0,TCGA-29-1696,1.0
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0,0.907308,1.0,TCGA-13-1483,1.0
TCGA-61-1738,TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0,0.151342,1.0,TCGA-61-1738,0.0,0.849077,1.0,TCGA-61-1738,1.0
TCGA-25-2393,TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0,0.222550,1.0,TCGA-25-2393,0.0,0.788705,1.0,TCGA-25-2393,1.0
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0,0.917132,0.0,TCGA-29-2414,1.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0,0.487189,0.0,TCGA-24-1563,0.0


In [535]:
test_temp_merge['mean_predict'] = (test_temp_merge['predict_x'] + test_temp_merge['predict_y']*0.5 + test_temp_merge['predict_add']*0.5)/2

test_temp_merge

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y,predict_add,label_add,index_ID,predicted_prob_add,mean_predict
index_ID,,,,,,,,,,,,,,
TCGA-36-1580,TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0,0.083512,1.0,TCGA-36-1580,0.0,0.377572,1.0,TCGA-36-1580,0.0,0.460831
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0,0.105563,0.0,TCGA-29-1769,0.0,0.350427
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0,0.937590,1.0,TCGA-29-1703,1.0,0.832690
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0,0.975946,1.0,TCGA-29-1696,1.0,0.830708
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0,0.907308,1.0,TCGA-13-1483,1.0,0.689670
TCGA-61-1738,TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0,0.151342,1.0,TCGA-61-1738,0.0,0.849077,1.0,TCGA-61-1738,1.0,0.519566
TCGA-25-2393,TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0,0.222550,1.0,TCGA-25-2393,0.0,0.788705,1.0,TCGA-25-2393,1.0,0.527130
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0,0.917132,0.0,TCGA-29-2414,1.0,0.704009
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0,0.487189,0.0,TCGA-24-1563,0.0,0.491842


## Modification prediction!

In [541]:
mod_index = test_merge.loc[(test_merge['predict_y'] < 0.08) ]

mod_index

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y
index_ID,,,,,,,,,
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0


In [543]:
mod_index.ID[1]

'TCGA-24-1563'

In [571]:
test_p_1temp

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-23-1027,9.534943e-01,1.0,TCGA-23-1027,1.0
TCGA-36-1580,6.911207e-01,1.0,TCGA-36-1580,1.0
TCGA-29-1769,6.269382e-01,0.0,TCGA-29-1769,1.0
TCGA-23-1122,1.569815e-09,0.0,TCGA-23-1122,0.0
TCGA-24-1474,3.728391e-12,0.0,TCGA-24-1474,0.0
TCGA-10-0927,1.387491e-02,0.0,TCGA-10-0927,0.0
TCGA-61-1914,6.077649e-13,0.0,TCGA-61-1914,0.0
TCGA-29-1703,9.466208e-01,1.0,TCGA-29-1703,1.0
TCGA-04-1365,1.156189e-02,0.0,TCGA-04-1365,0.0


In [572]:
test_1_final = test_p_1temp

In [ ]:
#test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0

In [573]:
test_1_final.loc[test_1_final['ID'] == mod_index.ID[0]]

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0


In [574]:
test_1_final.loc[test_1_final['ID'] == mod_index.ID[0], 'predicted_prob'] = 0

In [575]:
test_1_final.loc[test_1_final['ID'] == mod_index.ID[1]]

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0


In [577]:
test_1_final.loc[test_1_final['ID'] == mod_index.ID[1], 'predicted_prob'] = 0

In [578]:
test_1_final.loc[test_1_final['ID'] == mod_index.ID[0]]

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-29-1769,0.626938,0.0,TCGA-29-1769,0.0


In [579]:
test_1_final.loc[test_1_final['ID'] == mod_index.ID[1]]

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-24-1563,0.714755,0.0,TCGA-24-1563,0.0


In [581]:
cnf = confusion_matrix(test_1_final['label'], test_1_final['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_1_final['label'], test_1_final['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[12  1]
 [ 4 25]]
Accuracy :  0.8809523809523809
Sensitivity :  0.9230769230769231
Specificity :  0.8620689655172413
<class 'numpy.ndarray'>
AUC:  0.8925729442970822


In [67]:
test_merge.loc[test_merge['mean_predict'] >= 0.5, 'mod_predicted_prob'] = 1
test_merge.loc[test_merge['mean_predict'] < 0.5, 'mod_predicted_prob'] = 0

In [68]:
test_merge

,ID,predict_x,label_x,ID_x,predicted_prob_x,predict_y,label_y,ID_y,predicted_prob_y,mean_predict,mod_predicted_prob
index_ID,,,,,,,,,,,
TCGA-36-1580,TCGA-36-1580,0.691121,1.0,TCGA-36-1580,1.0,0.083512,1.0,TCGA-36-1580,0.0,0.448077,0.0
TCGA-29-1769,TCGA-29-1769,0.626938,0.0,TCGA-29-1769,1.0,0.042268,0.0,TCGA-29-1769,0.0,0.393070,0.0
TCGA-29-1703,TCGA-29-1703,0.946621,1.0,TCGA-29-1703,1.0,0.499927,1.0,TCGA-29-1703,0.0,0.767943,1.0
TCGA-29-1696,TCGA-29-1696,0.954286,1.0,TCGA-29-1696,1.0,0.438316,1.0,TCGA-29-1696,0.0,0.747898,1.0
TCGA-13-1483,TCGA-13-1483,0.817453,1.0,TCGA-13-1483,1.0,0.216467,1.0,TCGA-13-1483,0.0,0.577059,1.0
TCGA-61-1738,TCGA-61-1738,0.538922,1.0,TCGA-61-1738,1.0,0.151342,1.0,TCGA-61-1738,0.0,0.383890,0.0
TCGA-25-2393,TCGA-25-2393,0.548632,1.0,TCGA-25-2393,1.0,0.222550,1.0,TCGA-25-2393,0.0,0.418199,0.0
TCGA-29-2414,TCGA-29-2414,0.805669,0.0,TCGA-29-2414,1.0,0.287566,0.0,TCGA-29-2414,0.0,0.598428,1.0
TCGA-24-1563,TCGA-24-1563,0.714755,0.0,TCGA-24-1563,1.0,0.050670,0.0,TCGA-24-1563,0.0,0.449121,0.0


# Transfer Learning

결론적으로 이미 잘 훈련된 모델이 있고, 특히 해당 모델과 유사한 문제를 해결시 transfer learining을 사용합니다.
실질적 조언
새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우

데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다.
새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.
새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우

새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.
새로 훈련할 데이터가 적으며 original 데이터와 다른 경우

데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.
새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우

데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

In [442]:
model.classifier2

Sequential(
  (0): Linear(in_features=100, out_features=250, bias=True)
  (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Dropout(p=0.25, inplace=False)
)

In [443]:
model_pre_temp = model

In [444]:
model_temp = DNN_seq_2()

In [445]:
for param in model_pre_temp.parameters():
    param.requires_grad = False

model_temp.classifier4 = model_pre_temp.classifier2    
#model_temp.classifier2 = model_pre_temp.classifier2


In [446]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
#model_2 = DNN_seq_2()   
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model_temp.parameters(), lr=0.001, weight_decay = 1e-5)
    
#model_2 = DNN_seq_2()


use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model_temp = model_temp.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_temp.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [447]:
predict_pre_1 = []

for epoch in range(num_epochs):
    start_time = time.time()
    model_temp.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model_temp(trn_X)
        for i in trn_pred:
            predict_pre_1.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.5815005898475647, Accuracy: 0.699999988079071 % 	 time=0.18s
******************** Train ********************
Loss: 0.6397777795791626, Accuracy: 0.7230769395828247 % 	 time=0.15s
******************** Train ********************
Loss: 0.7110729217529297, Accuracy: 0.7230769395828247 % 	 time=0.16s
******************** Train ********************
Loss: 0.4122587740421295, Accuracy: 0.7230769395828247 % 	 time=0.15s
******************** Train ********************
Loss: 0.6807829141616821, Accuracy: 0.7076923251152039 % 	 time=0.16s
******************** Train ********************
Loss: 0.4648997187614441, Accuracy: 0.7153846025466919 % 	 time=0.15s
******************** Train ********************
Loss: 0.5600745677947998, Accuracy: 0.7153846025466919 % 	 time=0.16s
******************** Train ********************
Loss: 0.34087198972702026, Accuracy: 0.7461538314819336 % 	 time=0.16s
******************** Train ********************
Loss: 0.

******************** Train ********************
Loss: 0.11002825200557709, Accuracy: 0.8692307472229004 % 	 time=0.16s
******************** Train ********************
Loss: 0.5053497552871704, Accuracy: 0.8999999761581421 % 	 time=0.15s
******************** Train ********************
Loss: 0.09194616973400116, Accuracy: 0.9384615421295166 % 	 time=0.16s
******************** Train ********************
Loss: 0.1149885281920433, Accuracy: 0.8999999761581421 % 	 time=0.15s
******************** Train ********************
Loss: 0.016813818365335464, Accuracy: 0.9307692050933838 % 	 time=0.15s
******************** Train ********************
Loss: 1.238769292831421, Accuracy: 0.8153846263885498 % 	 time=0.15s
******************** Train ********************
Loss: 0.6086644530296326, Accuracy: 0.9076923131942749 % 	 time=0.16s
******************** Train ********************
Loss: 0.20249898731708527, Accuracy: 0.8692307472229004 % 	 time=0.15s
******************** Train ********************
Loss

In [448]:
criterion = nn.BCELoss()

In [449]:
model_temp.eval()
#predict = []
#y_val = val_y
predict_2_temp = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_temp(val_X)
        for i in val_pred:
            predict_2_temp.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 0., 0.])
tensor([1, 0, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 1, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([1, 0, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.16241788864135742, Accuracy: 0.7111111283302307 %
**********************************************
Val accuracy:0.711


In [450]:
predict_ = pd.DataFrame(predict_2_temp)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.805975,1.0
0.454256,1.0
0.169648,0.0
0.010601,0.0
0.039155,0.0
0.543723,0.0
0.000800,0.0
0.697436,1.0
0.259694,0.0


In [451]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_2_temp)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)


test_p_2 = pd.concat([predict_.iloc[:,0], label, val_label, val_label], axis = 1)

#print(test_p)

test_p_2.columns = ['predict','label','index_ID', 'ID']
test_p_2.loc[test_p_2['predict'] >= 0.5, 'predicted_prob'] = 1
test_p_2.loc[test_p_2['predict'] < 0.5, 'predicted_prob'] = 0


test_p.columns = ['predict','label']
test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 9  4]
 [ 6 23]]
Accuracy :  0.7619047619047619
Sensitivity :  0.6923076923076923
Specificity :  0.7931034482758621
AUC:  0.7427055702917772


In [151]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.746858	 Accuracy:50.000%
Epoch : 0 [32/171 (18%)]	Loss: 0.662493	 Accuracy:60.417%
Epoch : 0 [64/171 (36%)]	Loss: 0.704260	 Accuracy:60.000%
Epoch : 0 [96/171 (55%)]	Loss: 0.674394	 Accuracy:66.071%
Epoch : 0 [128/171 (73%)]	Loss: 0.722818	 Accuracy:67.361%
Epoch : 0 [110/171 (91%)]	Loss: 0.599336	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.575366	 Accuracy:87.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.667310	 Accuracy:70.833%
Epoch : 1 [64/171 (36%)]	Loss: 0.673750	 Accuracy:68.750%
Epoch : 1 [96/171 (55%)]	Loss: 0.751240	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.740712	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.710288	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.641428	 Accuracy:62.500%
Epoch : 2 [32/171 (18%)]	Loss: 0.618767	 Accuracy:68.750%
Epoch : 2 [64/171 (36%)]	Loss: 0.619703	 Accuracy:65.000%
Epoch : 2 [96/171 (55%)]	Loss: 0.579154	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.667362	 Accuracy:66.667%
Epoch : 2 [110/

In [152]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5934396982192993, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


# Voting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier()),
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('lr', LogisticRegressionCV()),
    ('ridge', RidgeClassifier()),
]

# hard vote
hard_vote  = VotingClassifier(models, voting='hard')
hard_vote_cv = cross_validate(hard_vote, x_train, y_train, cv=k_fold)
hard_vote.fit(x_train, y_train)

# soft vote
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote_cv = cross_validate(soft_vote, x_train, y_train, cv=k_fold)
soft_vote.fit(x_train, y_train)


In [105]:
class DNN_seq_1(nn.Module):
    def __init__(self):
        super(DNN_seq_1, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.25),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(100, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.25),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250,1, bias=True),
            torch.nn.BatchNorm1d(1)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [476]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_2(nn.Module):
    def __init__(self):
        super(DNN_seq_2, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(100, 50, bias=True),
            torch.nn.BatchNorm1d(50),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(50, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),

            torch.nn.Linear(100, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        #x_out = self.classifier4(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [ ]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_2mod(nn.Module):
    def __init__(self):
        super(DNN_seq_2mod, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(100, 50, bias=True),
            torch.nn.BatchNorm1d(50),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(50, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier4 = nn.Sequential(
            torch.nn.Linear(100, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.classifier4(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [216]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_3(nn.Module):
    def __init__(self):
        super(DNN_seq_3, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(100, 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.5),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [506]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_new(nn.Module):
    def __init__(self):
        super(DNN_seq_new, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]),200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.30),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(200, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.30),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(100, 50, bias=True),
            torch.nn.BatchNorm1d(50),
            torch.nn.LeakyReLU(),
            nn.Dropout(0.30),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(50, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [507]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [508]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

## Training 1

In [509]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3

model_1 = DNN_seq_1()    

## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model_1.parameters(), lr=0.001, weight_decay = 1e-5)
    
model_1 = DNN_seq_1()
use_cuda = torch.cuda.is_available()
num_epochs = 70
    
if use_cuda:
    model = model.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_1.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [141]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model_1.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model_1(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.584441065788269, Accuracy: 0.6846153736114502 % 	 time=0.14s
******************** Train ********************
Loss: 0.37215694785118103, Accuracy: 0.7769230604171753 % 	 time=0.11s
******************** Train ********************
Loss: 0.2013917863368988, Accuracy: 0.8307692408561707 % 	 time=0.11s
******************** Train ********************
Loss: 0.1639670431613922, Accuracy: 0.892307698726654 % 	 time=0.13s
******************** Train ********************
Loss: 0.31819847226142883, Accuracy: 0.9076923131942749 % 	 time=0.12s
******************** Train ********************
Loss: 0.07631389796733856, Accuracy: 0.8999999761581421 % 	 time=0.11s
******************** Train ********************
Loss: 0.11699125915765762, Accuracy: 0.9538461565971375 % 	 time=0.11s
******************** Train ********************
Loss: 0.05854858085513115, Accuracy: 0.9538461565971375 % 	 time=0.11s
******************** Train ********************
Loss:

## Training 2

In [445]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
model_2 = DNN_seq_2()   
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model_2.parameters(), lr=0.001, weight_decay = 1e-5)
    
model_2 = DNN_seq_2()


use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model_2 = model_2.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_2.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [241]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model_2(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.8320714831352234, Accuracy: 0.6666666865348816 % 	 time=0.23s
******************** Train ********************
Loss: 0.5061639547348022, Accuracy: 0.6909090876579285 % 	 time=0.21s
******************** Train ********************
Loss: 0.5217592120170593, Accuracy: 0.6909090876579285 % 	 time=0.21s
******************** Train ********************
Loss: 0.688666582107544, Accuracy: 0.7151514887809753 % 	 time=0.21s
******************** Train ********************
Loss: 0.30240315198898315, Accuracy: 0.7030302882194519 % 	 time=0.21s
******************** Train ********************
Loss: 0.4755634665489197, Accuracy: 0.7272727489471436 % 	 time=0.21s
******************** Train ********************
Loss: 0.3247077167034149, Accuracy: 0.7151514887809753 % 	 time=0.21s
******************** Train ********************
Loss: 0.5992145538330078, Accuracy: 0.7030302882194519 % 	 time=0.21s
******************** Train ********************
Loss: 1.

******************** Train ********************
Loss: 0.5444689393043518, Accuracy: 0.8909090757369995 % 	 time=0.21s
******************** Train ********************
Loss: 0.86505526304245, Accuracy: 0.903030276298523 % 	 time=0.21s
******************** Train ********************
Loss: 0.2206982672214508, Accuracy: 0.8969696760177612 % 	 time=0.21s
******************** Train ********************
Loss: 1.9621505737304688, Accuracy: 0.903030276298523 % 	 time=0.21s
******************** Train ********************
Loss: 0.19641944766044617, Accuracy: 0.8909090757369995 % 	 time=0.21s
******************** Train ********************
Loss: 0.06559957563877106, Accuracy: 0.9090909361839294 % 	 time=0.21s
******************** Train ********************
Loss: 0.02798665501177311, Accuracy: 0.903030276298523 % 	 time=0.21s
******************** Train ********************
Loss: 0.08178630471229553, Accuracy: 0.9212121367454529 % 	 time=0.22s
******************** Train ********************
Loss: 0.0

## Training 3

In [510]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
model_3 = DNN_seq_new()   
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model_3.parameters(), lr=0.001, weight_decay = 1e-5)
    
model_3 = DNN_seq_new()


##
use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model_3 = model_3.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_3.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [511]:


#predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model_3(trn_X)
        #for i in trn_pred:
        #    predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.6863959431648254, Accuracy: 0.6181818246841431 % 	 time=0.20s
******************** Train ********************
Loss: 0.3292355537414551, Accuracy: 0.7272727489471436 % 	 time=0.16s
******************** Train ********************
Loss: 0.562929093837738, Accuracy: 0.7696969509124756 % 	 time=0.19s
******************** Train ********************
Loss: 0.4331723153591156, Accuracy: 0.800000011920929 % 	 time=0.16s
******************** Train ********************
Loss: 0.30614250898361206, Accuracy: 0.8666666746139526 % 	 time=0.17s
******************** Train ********************
Loss: 0.4113381505012512, Accuracy: 0.8909090757369995 % 	 time=0.16s
******************** Train ********************
Loss: 0.15512089431285858, Accuracy: 0.9151515364646912 % 	 time=0.17s
******************** Train ********************
Loss: 0.3054869771003723, Accuracy: 0.9090909361839294 % 	 time=0.16s
******************** Train ********************
Loss: 0.

******************** Train ********************
Loss: 0.09617182612419128, Accuracy: 0.9454545378684998 % 	 time=0.17s
******************** Train ********************
Loss: 0.02341035008430481, Accuracy: 0.9333333373069763 % 	 time=0.16s
******************** Train ********************
Loss: 0.0630071833729744, Accuracy: 0.9575757384300232 % 	 time=0.21s
******************** Train ********************
Loss: 0.9255126714706421, Accuracy: 0.9454545378684998 % 	 time=0.19s
******************** Train ********************
Loss: 0.01234782487154007, Accuracy: 0.9212121367454529 % 	 time=0.18s
******************** Train ********************
Loss: 0.002612736541777849, Accuracy: 0.9696969985961914 % 	 time=0.20s
******************** Train ********************
Loss: 0.03966129943728447, Accuracy: 0.9818181991577148 % 	 time=0.20s
******************** Train ********************
Loss: 0.07900598645210266, Accuracy: 0.9515151381492615 % 	 time=0.19s
******************** Train ********************
L

## Voting Test

## Test 1

In [142]:
criterion = nn.BCELoss()

In [143]:
model_1.eval()
#predict = []
#y_val = val_y
predict_1 = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_1(val_X)
        for i in val_pred:
            predict_1.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 0., 0.])
tensor([1, 0, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 1, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 1, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([1, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 1, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([1, 0, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.1342855840921402, Accuracy: 0.6888889074325562 %
**********************************************
Val accuracy:0.689


In [144]:
predict_ = pd.DataFrame(predict_1)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
9.762787e-01,1.0
7.681830e-02,1.0
3.301538e-01,0.0
1.546975e-07,0.0
6.442643e-06,0.0
7.398000e-01,0.0
2.629387e-08,0.0
8.875772e-01,1.0
1.526865e-01,0.0


In [145]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_1)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[10  3]
 [ 8 21]]
Accuracy :  0.7380952380952381
Sensitivity :  0.7692307692307693
Specificity :  0.7241379310344828
AUC:  0.746684350132626


## Test 2

In [477]:
criterion = nn.BCELoss()

In [478]:
model_load = 1

In [479]:
if (model_load == 1):
    model_2 = DNN_seq_2()
    #model = TheModelClass(*args, **kwargs)                                                                                                     
    #optimizer = TheOptimizerClass(*args, **kwargs)                                                                                             
    checkpoint = torch.load("./platin_model_save/platin_model_250_50_100_model_1_100_250_Semi_Final_Voting_test_2.pth")                                                                                  
    model_2.load_state_dict(checkpoint['model_state_dict'])                                                                                         
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
    #num_epochs = checkpoint['epoch']                                                                                                           
    loss = checkpoint['loss']                                                                                                                   
    #model.eval() 
    print('model_load check')

model_load check


In [480]:
model_2.eval()
#predict = []
#y_val = val_y
predict_2 = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_2(val_X)
        for i in val_pred:
            predict_2.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 0., 0.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.021410737186670303, Accuracy: 0.7333333492279053 %
**********************************************
Val accuracy:0.733


In [481]:
predict_ = pd.DataFrame(predict_2)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
5.568392e-01,1.0
8.307219e-02,1.0
4.053841e-02,0.0
4.673898e-04,0.0
7.432269e-07,0.0
1.467769e-01,0.0
4.366220e-08,0.0
5.193180e-01,1.0
6.399535e-02,0.0


In [482]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_2)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)


test_p_2 = pd.concat([predict_.iloc[:,0], label, val_label, val_label], axis = 1)

#print(test_p)

test_p_2.columns = ['predict','label','index_ID', 'ID']
test_p_2.loc[test_p_2['predict'] >= 0.5, 'predicted_prob'] = 1
test_p_2.loc[test_p_2['predict'] < 0.5, 'predicted_prob'] = 0


test_p.columns = ['predict','label']
test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 5  8]
 [ 1 28]]
Accuracy :  0.7857142857142857
Sensitivity :  0.38461538461538464
Specificity :  0.9655172413793104
AUC:  0.6750663129973474


In [41]:
cnf = confusion_matrix(test_p_2['label'], test_p_2['predicted_prob'], labels = [1,0])

print(test_p_2.set_index('index_ID'))

test_p_2temp = test_p_2.set_index('index_ID')


print(cnf)
###print(pd.DataFrame(test_p_2))
is_index = test_p_2temp['predicted_prob'] == 0.0

#print(is_index)

#print(test_p_2.loc[test_p_2['predict'] < 0.5, 'predicted_prob'])
test_2 = test_p_2temp[test_p_2temp['predicted_prob'] == 0.0]
#test_p_temp = test_p_2.drop['ID']

#print(test_p_2)
test_2
#mask = test_p_2[test_p_2['label'].isin(0)]

#test_p_2[~mask.astype(int)].head()

#mask = mock_data['country'].isin(['Afghanistan', 'Nigeria']) mock_data[~mask].head()

#출처: https://note.espriter.net/1325 [espriter]

                   predict  label            ID  predicted_prob
index_ID                                                       
TCGA-23-1027  5.351465e-01    1.0  TCGA-23-1027             1.0
TCGA-36-1580  8.351165e-02    1.0  TCGA-36-1580             0.0
TCGA-29-1769  4.226791e-02    0.0  TCGA-29-1769             0.0
TCGA-23-1122  5.597841e-04    0.0  TCGA-23-1122             0.0
TCGA-24-1474  9.908041e-07    0.0  TCGA-24-1474             0.0
TCGA-10-0927  1.434996e-01    0.0  TCGA-10-0927             0.0
TCGA-61-1914  5.851045e-08    0.0  TCGA-61-1914             0.0
TCGA-29-1703  4.999270e-01    1.0  TCGA-29-1703             0.0
TCGA-04-1365  6.453138e-02    0.0  TCGA-04-1365             0.0
TCGA-29-1696  4.383163e-01    1.0  TCGA-29-1696             0.0
TCGA-13-1483  2.164667e-01    1.0  TCGA-13-1483             0.0
TCGA-61-1738  1.513424e-01    1.0  TCGA-61-1738             0.0
TCGA-61-1911  8.708345e-04    0.0  TCGA-61-1911             0.0
TCGA-25-2393  2.225501e-01    1.0  TCGA-

,predict,label,ID,predicted_prob
index_ID,,,,
TCGA-36-1580,8.351165e-02,1.0,TCGA-36-1580,0.0
TCGA-29-1769,4.226791e-02,0.0,TCGA-29-1769,0.0
TCGA-23-1122,5.597841e-04,0.0,TCGA-23-1122,0.0
TCGA-24-1474,9.908041e-07,0.0,TCGA-24-1474,0.0
TCGA-10-0927,1.434996e-01,0.0,TCGA-10-0927,0.0
TCGA-61-1914,5.851045e-08,0.0,TCGA-61-1914,0.0
TCGA-29-1703,4.999270e-01,1.0,TCGA-29-1703,0.0
TCGA-04-1365,6.453138e-02,0.0,TCGA-04-1365,0.0
TCGA-29-1696,4.383163e-01,1.0,TCGA-29-1696,0.0


In [184]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model_2.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_250_50_100_model_1_100_250_Semi_Final_Voting_test_2__.pth")     

## Test 3

In [512]:
criterion = nn.BCELoss()

In [513]:
model_3.eval()
#predict = []
#y_val = val_y
predict_3 = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_3(val_X)
        for i in val_pred:
            predict_3.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 0., 0.])
tensor([0, 0, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 1, 0, 1, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 1, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 1., 0.])
tensor([1, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1., 1., 0., 0., 1.])
tensor([0, 0], dtype=torch.uint8)
tensor([0., 0.])
******************** Test ********************
Loss: 0.10455100238323212, Accuracy: 0.7555555701255798 %
**********************************************
Val accuracy:0.756


In [514]:
predict_ = pd.DataFrame(predict_3)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
9.615111e-01,1.0
3.775721e-01,1.0
1.055634e-01,0.0
5.133583e-07,0.0
1.891717e-09,0.0
4.357328e-01,0.0
7.614696e-09,0.0
9.375903e-01,1.0
8.562330e-02,0.0


In [522]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_3)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p_3 = pd.concat([predict_.iloc[:,0], label, val_label, val_label], axis = 1)

#print(test_p)

test_p_3.columns = ['predict_add','label_add','index_ID','ID']
test_p_3.loc[test_p_3['predict_add'] >= 0.5, 'predicted_prob_add'] = 1
test_p_3.loc[test_p_3['predict_add'] < 0.5, 'predicted_prob_add'] = 0


test_p.columns = ['predict','label']
test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[10  3]
 [ 5 24]]
Accuracy :  0.8095238095238095
Sensitivity :  0.7692307692307693
Specificity :  0.8275862068965517
AUC:  0.7984084880636604


In [524]:
cnf = confusion_matrix(test_p_3['label_add'], test_p_3['predicted_prob_add'], labels = [1,0])

print(test_p_3.set_index('ID'))

test_p_3temp = test_p_3.set_index('ID')


print(cnf)
###print(pd.DataFrame(test_p_2))
###is_index = test_p_3temp['predicted_prob'] == 1.0

#print(is_index)

#print(test_p_2.loc[test_p_2['predict'] < 0.5, 'predicted_prob'])
test_3 = test_p_3temp
#test_p_temp = test_p_2.drop['ID']

#print(test_p_2)
test_3
#mask = test_p_2[test_p_2['label'].isin(0)]

#test_p_2[~mask.astype(int)].head()

#mask = mock_data['country'].isin(['Afghanistan', 'Nigeria']) mock_data[~mask].head()

#출처: https://note.espriter.net/1325 [espriter]

               predict_add  label_add      index_ID  predicted_prob_add
ID                                                                     
TCGA-23-1027  9.615111e-01        1.0  TCGA-23-1027                 1.0
TCGA-36-1580  3.775721e-01        1.0  TCGA-36-1580                 0.0
TCGA-29-1769  1.055634e-01        0.0  TCGA-29-1769                 0.0
TCGA-23-1122  5.133583e-07        0.0  TCGA-23-1122                 0.0
TCGA-24-1474  1.891717e-09        0.0  TCGA-24-1474                 0.0
TCGA-10-0927  4.357328e-01        0.0  TCGA-10-0927                 0.0
TCGA-61-1914  7.614696e-09        0.0  TCGA-61-1914                 0.0
TCGA-29-1703  9.375903e-01        1.0  TCGA-29-1703                 1.0
TCGA-04-1365  8.562330e-02        0.0  TCGA-04-1365                 0.0
TCGA-29-1696  9.759462e-01        1.0  TCGA-29-1696                 1.0
TCGA-13-1483  9.073076e-01        1.0  TCGA-13-1483                 1.0
TCGA-61-1738  8.490772e-01        1.0  TCGA-61-1738             

,predict_add,label_add,index_ID,predicted_prob_add
ID,,,,
TCGA-23-1027,9.615111e-01,1.0,TCGA-23-1027,1.0
TCGA-36-1580,3.775721e-01,1.0,TCGA-36-1580,0.0
TCGA-29-1769,1.055634e-01,0.0,TCGA-29-1769,0.0
TCGA-23-1122,5.133583e-07,0.0,TCGA-23-1122,0.0
TCGA-24-1474,1.891717e-09,0.0,TCGA-24-1474,0.0
TCGA-10-0927,4.357328e-01,0.0,TCGA-10-0927,0.0
TCGA-61-1914,7.614696e-09,0.0,TCGA-61-1914,0.0
TCGA-29-1703,9.375903e-01,1.0,TCGA-29-1703,1.0
TCGA-04-1365,8.562330e-02,0.0,TCGA-04-1365,0.0


In [175]:
Final_test = pd.concat([test_1, test_2], axis = 0)

In [177]:
print(Final_test.shape[0])

print(Final_test)

cnf = confusion_matrix(Final_test['label'], Final_test['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)


auc  =  round(roc_auc_score(test_p['label'],test_p['predicted_prob']),4)

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

#print('AUC: ', auc(fpr, trp))
print('AUC: ', auc)

42
                   predict  label  predicted_prob
ID                                               
TCGA-23-1027  9.534943e-01    1.0             1.0
TCGA-36-1580  6.911207e-01    1.0             1.0
TCGA-29-1703  9.466208e-01    1.0             1.0
TCGA-29-1696  9.542856e-01    1.0             1.0
TCGA-13-1483  8.174533e-01    1.0             1.0
TCGA-61-1738  5.389222e-01    1.0             1.0
TCGA-25-2393  5.486322e-01    1.0             1.0
TCGA-13-0893  8.634816e-01    1.0             1.0
TCGA-13-0795  9.955494e-01    1.0             1.0
TCGA-25-1628  6.765153e-01    1.0             1.0
TCGA-09-0366  9.845570e-01    1.0             1.0
TCGA-24-1924  1.748448e-05    1.0             0.0
TCGA-61-1733  9.637365e-01    1.0             1.0
TCGA-29-1769  4.053841e-02    0.0             0.0
TCGA-23-1122  4.673898e-04    0.0             0.0
TCGA-24-1474  7.432269e-07    0.0             0.0
TCGA-10-0927  1.467769e-01    0.0             0.0
TCGA-61-1914  4.366220e-08    0.0             0

In [223]:
predict_1

[array([0.96523094], dtype=float32),
 array([0.40869933], dtype=float32),
 array([0.18736918], dtype=float32),
 array([6.5656457e-07], dtype=float32),
 array([2.3786907e-08], dtype=float32),
 array([0.05763014], dtype=float32),
 array([1.9820663e-10], dtype=float32),
 array([0.9488913], dtype=float32),
 array([0.41917148], dtype=float32),
 array([0.9683527], dtype=float32),
 array([0.75444454], dtype=float32),
 array([0.69513965], dtype=float32),
 array([0.00034798], dtype=float32),
 array([0.70198876], dtype=float32),
 array([2.795339e-09], dtype=float32),
 array([6.0754446e-05], dtype=float32),
 array([0.91316634], dtype=float32),
 array([0.6346924], dtype=float32),
 array([5.5596742e-12], dtype=float32),
 array([1.352942e-13], dtype=float32),
 array([0.5749145], dtype=float32),
 array([0.8809959], dtype=float32),
 array([2.519986e-05], dtype=float32),
 array([0.00343669], dtype=float32),
 array([0.49783298], dtype=float32),
 array([0.71625566], dtype=float32),
 array([0.78362393], d

In [224]:
predict_2

[array([0.55683917], dtype=float32),
 array([0.08307219], dtype=float32),
 array([0.04053841], dtype=float32),
 array([0.00046739], dtype=float32),
 array([7.432269e-07], dtype=float32),
 array([0.14677685], dtype=float32),
 array([4.3662197e-08], dtype=float32),
 array([0.51931804], dtype=float32),
 array([0.06399535], dtype=float32),
 array([0.45912912], dtype=float32),
 array([0.2208362], dtype=float32),
 array([0.15557681], dtype=float32),
 array([0.00075173], dtype=float32),
 array([0.22724117], dtype=float32),
 array([4.1943338e-08], dtype=float32),
 array([0.01102896], dtype=float32),
 array([0.2967533], dtype=float32),
 array([0.04953551], dtype=float32),
 array([7.937189e-09], dtype=float32),
 array([1.4820315e-06], dtype=float32),
 array([0.24651052], dtype=float32),
 array([0.4239356], dtype=float32),
 array([0.00011356], dtype=float32),
 array([0.03767242], dtype=float32),
 array([0.19217741], dtype=float32),
 array([0.07303619], dtype=float32),
 array([0.20627727], dtype=f

In [225]:
predict_3

[array([0.8607451], dtype=float32),
 array([0.4790696], dtype=float32),
 array([0.3565798], dtype=float32),
 array([7.1915606e-06], dtype=float32),
 array([0.0014108], dtype=float32),
 array([0.16123456], dtype=float32),
 array([1.3273825e-07], dtype=float32),
 array([0.7402205], dtype=float32),
 array([0.3974231], dtype=float32),
 array([0.8689314], dtype=float32),
 array([0.6825226], dtype=float32),
 array([0.5898785], dtype=float32),
 array([0.00829941], dtype=float32),
 array([0.51559234], dtype=float32),
 array([0.00072743], dtype=float32),
 array([0.02738246], dtype=float32),
 array([0.69638824], dtype=float32),
 array([0.3023329], dtype=float32),
 array([3.455946e-05], dtype=float32),
 array([7.268803e-07], dtype=float32),
 array([0.5242045], dtype=float32),
 array([0.7060348], dtype=float32),
 array([0.08810126], dtype=float32),
 array([0.10685713], dtype=float32),
 array([0.4980478], dtype=float32),
 array([0.42031947], dtype=float32),
 array([0.642132], dtype=float32),
 array

# Combination

In [339]:
import operator

predict_ = []

for i in predict:
    #print (i) # 3
    predict_.append(i) # 3
    #print (predict_)

predict_1_2 = []

for i in predict:
    #print (i) # 3
    predict_1_2.append(i) # 3
    #print (predict_2_)

predict_3_ = []

for i in predict_3:
    #print (i) # 3
    predict_3_.append(i) # 3
    #print (predict_2_)

#predict_ = map(operator.mul, predict, 0.7)
#predict_2_ = map(operator.mul, predict_2, 0.3)
#predict_total = map(operator.add, predict_3, predict)
predict_total = map(operator.add, predict_, predict_3_)
#predict_total = map(operator.add, predict_total, predict_1_2)
print(predict_total)

##for i in predict_total:
##    print (i) # 3
    #predict_final.append(i) # 3
    #print (predict_final)


In [340]:
predict_final = []

for i in predict_total:
    print (i) # 3
    predict_final.append(i/3) # 3
    print (predict_final)



[1.8450831]
[array([0.6150277], dtype=float32)]
[1.1708615]
[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32)]
[0.64521825]
[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32), array([0.21507275], dtype=float32)]
[0.6269195]
[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32), array([0.21507275], dtype=float32), array([0.20897317], dtype=float32)]
[0.68808395]
[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32), array([0.21507275], dtype=float32), array([0.20897317], dtype=float32), array([0.22936131], dtype=float32)]
[0.7017898]
[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32), array([0.21507275], dtype=float32), array([0.20897317], dtype=float32), array([0.22936131], dtype=float32), array([0.23392993], dtype=float32)]
[0.6526075]
[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32), array([0.21507275], dtype=float32), array([0.20897317], dtype=float32), array([0.22936

In [341]:
print(predict_final)
type(predict_final)
len(predict_final)

[array([0.6150277], dtype=float32), array([0.39028716], dtype=float32), array([0.21507275], dtype=float32), array([0.20897317], dtype=float32), array([0.22936131], dtype=float32), array([0.23392993], dtype=float32), array([0.21753584], dtype=float32), array([0.5265177], dtype=float32), array([0.20570122], dtype=float32), array([0.5792962], dtype=float32), array([0.44329524], dtype=float32), array([0.49143684], dtype=float32), array([0.20946658], dtype=float32), array([0.29925227], dtype=float32), array([0.25958216], dtype=float32), array([0.1314901], dtype=float32), array([0.430526], dtype=float32), array([0.27026558], dtype=float32), array([0.26973876], dtype=float32), array([0.25645262], dtype=float32), array([0.32059684], dtype=float32), array([0.46076044], dtype=float32), array([0.22350793], dtype=float32), array([0.18967116], dtype=float32), array([0.32009074], dtype=float32), array([0.32174382], dtype=float32), array([0.43135262], dtype=float32), array([0.21623378], dtype=float32

42

## Evaluating

In [342]:
predict_ = pd.DataFrame(predict_final)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.615028,1.0
0.390287,1.0
0.215073,0.0
0.208973,0.0
0.229361,0.0
0.233930,0.0
0.217536,0.0
0.526518,1.0
0.205701,0.0


In [343]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict_final)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 6  7]
 [ 1 28]]
Accuracy :  0.8095238095238095
Sensitivity :  0.46153846153846156
Specificity :  0.9655172413793104
AUC:  0.7135278514588861


In [76]:
model_1 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_1 = model_1.cuda()
    
fit(model_1, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.935273	 Accuracy:68.750%
Epoch : 0 [32/171 (18%)]	Loss: 0.791408	 Accuracy:70.833%
Epoch : 0 [64/171 (36%)]	Loss: 0.752347	 Accuracy:68.750%
Epoch : 0 [96/171 (55%)]	Loss: 0.723812	 Accuracy:63.393%
Epoch : 0 [128/171 (73%)]	Loss: 0.811310	 Accuracy:65.972%
Epoch : 0 [110/171 (91%)]	Loss: 0.819974	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.823073	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.663486	 Accuracy:64.583%
Epoch : 1 [64/171 (36%)]	Loss: 0.554099	 Accuracy:66.250%
Epoch : 1 [96/171 (55%)]	Loss: 0.667068	 Accuracy:66.071%
Epoch : 1 [128/171 (73%)]	Loss: 0.624170	 Accuracy:65.278%
Epoch : 1 [110/171 (91%)]	Loss: 0.637554	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.651228	 Accuracy:75.000%
Epoch : 2 [32/171 (18%)]	Loss: 0.534523	 Accuracy:81.250%
Epoch : 2 [64/171 (36%)]	Loss: 0.607727	 Accuracy:71.250%
Epoch : 2 [96/171 (55%)]	Loss: 0.625725	 Accuracy:69.643%
Epoch : 2 [128/171 (73%)]	Loss: 0.518599	 Accuracy:68.750%
Epoch : 2 [110/

In [77]:
model_2 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_2 = model_2.cuda()
    
fit(model_2, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.745150	 Accuracy:56.250%
Epoch : 0 [32/171 (18%)]	Loss: 0.593750	 Accuracy:70.833%
Epoch : 0 [64/171 (36%)]	Loss: 0.609681	 Accuracy:72.500%
Epoch : 0 [96/171 (55%)]	Loss: 0.811276	 Accuracy:70.536%
Epoch : 0 [128/171 (73%)]	Loss: 0.909858	 Accuracy:68.750%
Epoch : 0 [110/171 (91%)]	Loss: 0.555978	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.671668	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.751559	 Accuracy:60.417%
Epoch : 1 [64/171 (36%)]	Loss: 0.629498	 Accuracy:66.250%
Epoch : 1 [96/171 (55%)]	Loss: 0.660346	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.670158	 Accuracy:68.056%
Epoch : 1 [110/171 (91%)]	Loss: 0.560989	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.624824	 Accuracy:68.750%
Epoch : 2 [32/171 (18%)]	Loss: 0.593675	 Accuracy:70.833%
Epoch : 2 [64/171 (36%)]	Loss: 0.510769	 Accuracy:67.500%
Epoch : 2 [96/171 (55%)]	Loss: 0.542457	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.449139	 Accuracy:68.056%
Epoch : 2 [110/

In [78]:
criterion = nn.BCELoss()

In [79]:
model_1.eval()
predict_1 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_1(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_1.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5453603863716125, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [80]:
criterion = nn.BCELoss()

In [81]:
model_2.eval()
predict_2 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_2(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_2.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.586574375629425, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [69]:
predict_1

[array([0.39712372], dtype=float32),
 array([0.20998392], dtype=float32),
 array([0.32124352], dtype=float32),
 array([0.34580824], dtype=float32),
 array([0.43426463], dtype=float32),
 array([0.43412134], dtype=float32),
 array([0.39131775], dtype=float32),
 array([0.38226905], dtype=float32),
 array([0.42771617], dtype=float32),
 array([0.2733672], dtype=float32),
 array([0.42271107], dtype=float32),
 array([0.4628345], dtype=float32),
 array([0.4519838], dtype=float32),
 array([0.3947924], dtype=float32),
 array([0.39574006], dtype=float32),
 array([0.4404809], dtype=float32),
 array([0.3974393], dtype=float32),
 array([0.48102662], dtype=float32),
 array([0.42520103], dtype=float32),
 array([0.43733662], dtype=float32),
 array([0.3043311], dtype=float32),
 array([0.4567577], dtype=float32),
 array([0.36690694], dtype=float32),
 array([0.38817948], dtype=float32),
 array([0.5215086], dtype=float32),
 array([0.4337628], dtype=float32),
 array([0.40996504], dtype=float32),
 array([0.6

In [70]:
predict_2

[array([0.40936574], dtype=float32),
 array([0.30449998], dtype=float32),
 array([0.37039232], dtype=float32),
 array([0.37293124], dtype=float32),
 array([0.45267642], dtype=float32),
 array([0.44899222], dtype=float32),
 array([0.41215253], dtype=float32),
 array([0.40887272], dtype=float32),
 array([0.4442977], dtype=float32),
 array([0.39285985], dtype=float32),
 array([0.4360027], dtype=float32),
 array([0.48552424], dtype=float32),
 array([0.46931365], dtype=float32),
 array([0.40514487], dtype=float32),
 array([0.41182923], dtype=float32),
 array([0.42791894], dtype=float32),
 array([0.405836], dtype=float32),
 array([0.4981986], dtype=float32),
 array([0.43440622], dtype=float32),
 array([0.44471112], dtype=float32),
 array([0.37168515], dtype=float32),
 array([0.47441038], dtype=float32),
 array([0.3651678], dtype=float32),
 array([0.40067512], dtype=float32),
 array([0.5199345], dtype=float32),
 array([0.4233134], dtype=float32),
 array([0.4351335], dtype=float32),
 array([0.

In [71]:
#predict_3 = predict_1 + predict_2

In [72]:
len(predict_3)

NameError: name 'predict_3' is not defined

In [56]:
type(predict_1)

list

In [92]:
import operator

predict_3 = map(operator.add, predict_1, predict_2)

In [93]:
print(predict_3)

In [94]:
predict = []

for i in predict_3:
    print (i/2)
    predict.append(i/2)
    print (predict)



[0.43645966]
[array([0.43645966], dtype=float32)]
[0.34106642]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32)]
[0.4075713]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32)]
[0.40122908]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32)]
[0.44226646]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32)]
[0.44394046]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32)]
[0.41578606]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0

In [97]:
print(predict)
type(predict)

[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32), array([0.41578606], dtype=float32), array([0.41548812], dtype=float32), array([0.4436235], dtype=float32), array([0.357646], dtype=float32), array([0.43614274], dtype=float32), array([0.4559007], dtype=float32), array([0.44684273], dtype=float32), array([0.42056525], dtype=float32), array([0.42621326], dtype=float32), array([0.42557824], dtype=float32), array([0.4176554], dtype=float32), array([0.45774207], dtype=float32), array([0.43780085], dtype=float32), array([0.4308337], dtype=float32), array([0.4050982], dtype=float32), array([0.4504068], dtype=float32), array([0.39789623], dtype=float32), array([0.41764146], dtype=float32), array([0.4750628], dtype=float32), array([0.4315881], dtype=float32), array([0.4379266], dtype=float32), array([0.4739846], dtype=float32), arra

list

In [104]:
val_y_pd_ = val_y_pd.to_numpy()
val_y_pd_ = torch.from_numpy(val_y_pd_)
val_y_pd_ = val_y_pd_.type_as(torch.FloatTensor())

In [ ]:
val_loss = criterion(predict, val_y)
val_loss_summary += val_loss
predicted = torch.max(val_pred.data, 1)[1] 
correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

In [96]:
val_y_pd

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    1.0
27    0.0
28    0.0
29    1.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    1.0
37    1.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
Name: label, dtype: float64

In [93]:
#num_ftrs = model.forward.in_features
 
for name, param in model.named_parameters():
    print (name, param.data)

layer1.0.weight tensor([[-0.0565, -0.1205,  0.0296,  ...,  0.2230, -0.0025,  0.0114],
        [-0.0879, -0.0202, -0.0235,  ...,  0.0587,  0.0370, -0.0111],
        [-0.0381, -0.0511, -0.0141,  ..., -0.0142,  0.0329,  0.0550],
        ...,
        [ 0.0349, -0.0490, -0.0410,  ..., -0.0243, -0.0495,  0.0048],
        [ 0.0487,  0.1016, -0.0085,  ..., -0.1036,  0.0225, -0.0505],
        [-0.0129,  0.1087,  0.0454,  ..., -0.1197,  0.0375,  0.0101]])
layer1.0.bias tensor([-0.0296, -0.0339,  0.0066, -0.0166, -0.0347, -0.0489,  0.0233,  0.0351,
         0.0348,  0.0116,  0.0404, -0.0197, -0.0154,  0.0126,  0.0236,  0.0361,
        -0.0178, -0.0001, -0.0028, -0.0274, -0.0495, -0.0317,  0.0020,  0.0040,
         0.0019,  0.0077,  0.0134, -0.0460,  0.0494,  0.0400, -0.0072, -0.0119,
         0.0044, -0.0115,  0.0237, -0.0051, -0.0136, -0.0101,  0.0195,  0.0419,
        -0.0333, -0.0286,  0.0231, -0.0157,  0.0131,  0.0034, -0.0038,  0.0113,
        -0.0342,  0.0022, -0.0403,  0.0045,  0.0368, -0.

In [95]:
model.layer1 = 

NameError: name 'Sequential' is not defined

In [79]:
nn.Linear(num_ftrs)

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [32]:
list(model.children())[0:3]

AttributeError: 'list' object has no attribute 'parameter'

## Train model

In [77]:
model = MLPRegressor()

NameError: name 'X_features' is not defined

In [78]:
criterion = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'model' is not defined

In [44]:
batch_size = 32

In [45]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = data_utils.TensorDataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

NameError: name 'val_X' is not defined

In [13]:
trn_X_pd = df2_mod.iloc[:,2:]

In [14]:
trn_X_pd

ID,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,ENSG00000109321,ENSG00000112499,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
TCGA-61-1910,0.970585,0.003561,0.044476,0.000000,0.000000,0.000000,0.043732,2.308592,1.014494,0.000000,...,0.000000,0.920597,0.000000,1.604301,0.299376,0.000000,0.894232,1.612154,0.761720,0.000000
TCGA-24-0982,0.395276,0.005010,0.090384,0.007062,0.000000,0.006438,0.082034,0.242661,0.691196,0.004156,...,24.797152,0.259035,4.625172,0.188089,4.211878,0.270091,0.034947,0.311310,0.357217,0.000000
TCGA-36-1580,1.403594,0.008641,0.133845,0.001970,0.002559,0.041762,0.074384,0.984290,2.329158,0.001391,...,4.017149,0.289083,0.893372,0.314861,4.817979,0.000000,0.000000,0.707253,15.996040,0.000000
TCGA-25-1321,0.308600,0.003129,0.004478,0.162223,0.000788,0.000000,0.101272,0.240456,1.740308,0.004819,...,2.576079,0.166843,0.000000,0.121147,1.356422,0.521892,0.157563,2.033770,1.495528,0.000000
TCGA-30-1892,0.206074,0.133216,0.119614,0.056072,0.000000,0.055381,0.280458,0.592872,0.914734,0.004400,...,1.058604,0.114269,2.825069,0.000000,3.716018,0.000000,0.000000,0.294278,1.890974,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,0.319288,0.000491,0.033696,0.005704,0.000988,0.011701,0.060742,0.663423,0.094262,0.000671,...,0.000000,0.000000,0.000000,3.038608,0.567030,0.000000,0.000000,0.071847,4.616723,0.000000
TCGA-24-0968,0.237050,0.009708,0.185222,0.000000,0.000000,0.000000,0.307004,0.213121,1.291373,0.003796,...,15.221570,0.525782,4.332951,2.720169,31.845646,0.205584,0.000000,0.710875,6.117782,0.000000
TCGA-30-1891,0.772034,0.000481,0.110194,0.000000,0.000646,0.200678,0.000000,0.098616,2.959271,0.077290,...,47.331214,0.000000,2.631492,0.000000,65.568652,0.000000,0.000000,0.000000,80.395413,0.000000
TCGA-09-0366,0.195435,0.000085,1.631170,0.026487,0.001376,0.006339,0.303848,0.703636,5.424154,0.008418,...,0.225023,0.534375,0.100086,1.481522,3.712525,0.000000,0.058985,2.477118,0.602935,0.001573


In [15]:
trn_y_pd = df2_mod.iloc[:,1]

In [16]:
trn_y_pd

TCGA-61-1910    0.007666
TCGA-24-0982    0.007190
TCGA-36-1580    0.027081
TCGA-25-1321    0.002315
TCGA-30-1892    0.057091
                  ...   
TCGA-04-1347    0.000000
TCGA-24-0968    0.013682
TCGA-30-1891    0.034186
TCGA-09-0366    0.032361
TCGA-13-0884    0.018597
Name: ENSG00000048545, Length: 214, dtype: float64

In [17]:
trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

In [18]:
batch_size=64

In [19]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

In [20]:
tmp = next(iter(trn_loader))

In [21]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)

In [22]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)